In [1]:
import pandas as pd

pd.set_option('display.max_columns', None)
df = pd.read_csv("./dataset/Loan_Default.csv")
df.head()


,ID,year,loan_limit,Gender,approv_in_adv,loan_type,loan_purpose,Credit_Worthiness,open_credit,business_or_commercial,loan_amount,rate_of_interest,Interest_rate_spread,Upfront_charges,term,Neg_ammortization,interest_only,lump_sum_payment,property_value,construction_type,occupancy_type,Secured_by,total_units,income,credit_type,Credit_Score,co-applicant_credit_type,age,submission_of_application,LTV,Region,Security_Type,Status,dtir1
0,24890,2019,cf,Sex Not Available,nopre,type1,p1,l1,nopc,nob/c,116500,NaN,NaN,NaN,360.0,not_neg,not_int,not_lpsm,118000.0,sb,pr,home,1U,1740.0,EXP,758,CIB,25-34,to_inst,98.728814,south,direct,1,45.0
1,24891,2019,cf,Male,nopre,type2,p1,l1,nopc,b/c,206500,NaN,NaN,NaN,360.0,not_neg,not_int,lpsm,NaN,sb,pr,home,1U,4980.0,EQUI,552,EXP,55-64,to_inst,NaN,North,direct,1,NaN
2,24892,2019,cf,Male,pre,type1,p1,l1,nopc,nob/c,406500,4.56,0.2000,595.0,360.0,neg_amm,not_int,not_lpsm,508000.0,sb,pr,home,1U,9480.0,EXP,834,CIB,35-44,to_inst,80.019685,south,direct,0,46.0
3,24893,2019,cf,Male,nopre,type1,p4,l1,nopc,nob/c,456500,4.25,0.6810,NaN,360.0,not_neg,not_int,not_lpsm,658000.0,sb,pr,home,1U,11880.0,EXP,587,CIB,45-54,not_inst,69.376900,North,direct,0,42.0
4,24894,2019,cf,Joint,pre,type1,p1,l1,nopc,nob/c,696500,4.00,0.3042,0.0,360.0,not_neg,not_int,not_lpsm,758000.0,sb,pr,home,1U,10440.0,CRIF,602,EXP,25-34,not_inst,91.886544,North,direct,0,39.0


In [2]:
df.isna().sum()
df.dtypes

ID                             int64
year                           int64
loan_limit                    object
Gender                        object
approv_in_adv                 object
loan_type                     object
loan_purpose                  object
Credit_Worthiness             object
open_credit                   object
business_or_commercial        object
loan_amount                    int64
rate_of_interest             float64
Interest_rate_spread         float64
Upfront_charges              float64
term                         float64
Neg_ammortization             object
interest_only                 object
lump_sum_payment              object
property_value               float64
construction_type             object
occupancy_type                object
Secured_by                    object
total_units                   object
income                       float64
credit_type                   object
Credit_Score                   int64
co-applicant_credit_type      object
a

In [3]:
y = df['Status']
X = df.drop(columns=['Status','Gender','ID'])
X.columns

Index(['year', 'loan_limit', 'approv_in_adv', 'loan_type', 'loan_purpose',
       'Credit_Worthiness', 'open_credit', 'business_or_commercial',
       'loan_amount', 'rate_of_interest', 'Interest_rate_spread',
       'Upfront_charges', 'term', 'Neg_ammortization', 'interest_only',
       'lump_sum_payment', 'property_value', 'construction_type',
       'occupancy_type', 'Secured_by', 'total_units', 'income', 'credit_type',
       'Credit_Score', 'co-applicant_credit_type', 'age',
       'submission_of_application', 'LTV', 'Region', 'Security_Type', 'dtir1'],
      dtype='object')

In [4]:
X = X.fillna(X.median(numeric_only=True))
X = pd.get_dummies(X, drop_first=True)
X.isna().sum()

year                                 0
loan_amount                          0
rate_of_interest                     0
Interest_rate_spread                 0
Upfront_charges                      0
term                                 0
property_value                       0
income                               0
Credit_Score                         0
LTV                                  0
dtir1                                0
loan_limit_ncf                       0
approv_in_adv_pre                    0
loan_type_type2                      0
loan_type_type3                      0
loan_purpose_p2                      0
loan_purpose_p3                      0
loan_purpose_p4                      0
Credit_Worthiness_l2                 0
open_credit_opc                      0
business_or_commercial_nob/c         0
Neg_ammortization_not_neg            0
interest_only_not_int                0
lump_sum_payment_not_lpsm            0
construction_type_sb                 0
occupancy_type_pr        

In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


In [6]:
# %pip install imbalanced-learn

from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=0)
X_resampled, Y_resampled = ros.fit_resample(X_train, y_train)

In [7]:
Y_resampled.value_counts()

Status
0    89625
1    89625
Name: count, dtype: int64

In [8]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(
    n_estimators=100,
    max_depth=5,   # limit tree depth
    random_state=42
)


# Train
rf.fit(X_resampled, Y_resampled)


RandomForestClassifier(max_depth=5, random_state=42)

In [9]:
y_pred= rf.predict(X_test)
y_pred

array([0, 0, 0, ..., 0, 0, 0])

In [10]:
rf.score(X_test, y_test)

0.9981166341561848

In [11]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred, target_names=[str(c) for c in rf.classes_]))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     22406
           1       0.99      1.00      1.00      7328

    accuracy                           1.00     29734
   macro avg       1.00      1.00      1.00     29734
weighted avg       1.00      1.00      1.00     29734



In [12]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

def evaluate(model, X_test, y_test):
    preds = model.predict(X_test)
    return {
        "Accuracy": accuracy_score(y_test, preds),
        "Precision": precision_score(y_test, preds),
        "Recall": recall_score(y_test, preds),
        "F1": f1_score(y_test, preds),
        "ROC-AUC": roc_auc_score(y_test, preds)
    }

print("Random Forest:", evaluate(rf, X_test, y_test))


Random Forest: {'Accuracy': 0.9981166341561848, 'Precision': 0.9924160346695557, 'Recall': 1.0, 'F1': 0.9961935834692768, 'ROC-AUC': 0.9987503347317682}


In [13]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(rf, X, y, cv=5, scoring="accuracy")
print("Cross-validation Accuracy:", scores.mean())


Cross-validation Accuracy: 0.9971211407815967


In [14]:
import joblib
joblib.dump(rf, 'best_loan_default_model.pkl')

['best_loan_default_model.pkl']

In [16]:
import joblib
joblib.dump(list(X.columns), 'X_columns.pkl')

['X_columns.pkl']